<a href="https://colab.research.google.com/github/Jaime44/WorkSpace/blob/main/models/content%20filter%20colaborative/content-based-recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import sys

# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
    GPU = False
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive
    import tensorflow as tf
    print("Versión de TensorFlow:", tf.__version__)
    print("Dispositivos disponibles:", tf.config.list_physical_devices())

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/Models/content filter colaborative'

    path_workspace = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)

    # Verificar la GPU
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        print('GPU no encontrada')
    else:
      print(f'Encontrada GPU: {device_name}')
      GPU = True

    # Habilitar la GPU para TensorFlow
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print('Memoria de la GPU configurada dinámicamente')
    else:
        print('No se encontraron dispositivos GPU configurables')
else:
    print("El código se está ejecutando en un entorno local.")
    path_workspace ='C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/'
    path_absolute = os.getcwd().replace("\\", "/")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/Models/content filter colaborative'

datasets_path = "/datasets/"
path_absolute = path_absolute+datasets_path


sys.path.append(path_workspace)

El código se está ejecutando en Google Colab.
Versión de TensorFlow: 2.15.0
Dispositivos disponibles: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Contenido de la carpeta en Google Drive:
['datasets', 'CF_The_age_of_recommeder_System_Original.ipynb', 'Content-Based_and_Tensorflow_Recommender_System_Original.ipynb', 'CF_The_age_of_recommeder_System.ipynb', 'Content-Based_and_Tensorflow_Recommender_System.ipynb', 'content-based-recommendation_Original.ipynb', 'movie-recommendation-system.ipynb', 'content-based-recommendation.ipynb', '00_CFC_Recommender_system_movie.ipynb', '00_CFC_Recommender_system_movieCOLAB.ipynb']
GPU no encontrada
No se encontraron dispositivos GPU configurables


![CBR](https://miro.medium.com/v2/resize:fit:1400/1*H_MMnrpLQrqTSJHdDOCMoA.png)

# What is Content Based Recommendation

Content-based recommendation, also known as content-based filtering, is a type of system or algorithm that provides recommendations to a user based on their interests and preferences. Those with such recommendation systems analyze the user's past preferences and likes, and suggest new items based on similar content.

Content-based recommendation analyzes the content of items and determines the ones that are suitable for the user based on similarity criteria. For example, when making a movie recommendation, the system can take into account the genres, actors, directors, and other features of the movies the user has liked or watched. Based on this information, the system suggests other movies with similar characteristics.

This recommendation system can utilize text analysis, tagging, categorization, or other content features along with the user profile or history to better understand the user's preferences. For instance, when making a music recommendation, the system can analyze features such as genres, instruments, tempo, and rhythm.

Content-based recommendation systems can be effective in providing personalized recommendations based on user preferences. The recommended items based on the user's past data can capture their interest and provide a better user experience.

# Business Problem
To recommend movies similar to the movies that a person who comes to our site to watch movies.

# Road Map

- 1. Creating the **TF-IDF Matrix**
- 2. Creation of **Cosine Similarity Matrix**
- 3. Making Recommendations Based on Similarities
- 4. Preparation of the Study Script


## Se cargan las librerías

In [74]:
# import Required Libraries

import pandas as pd
import numpy as np
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Se cargan los Datasets

In [4]:
# df = pd.read_csv(path_absolute+"df_mezclado_tags_ratings_movies_links_genTags.csv")

In [5]:
# dataFrame_onehot_encode = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs_onehot_encode.csv', sep=',')

# dataFrame_onehot_encode.head()


In [6]:
dataFrame_without_ohe = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs.csv')
print(dataFrame_without_ohe.shape)
dataFrame_without_ohe.head()

<ipython-input-6-c633e3f492c3>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dataFrame_without_ohe = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs.csv')


(25000095, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
0,1,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1147880044,5.0
1,3,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1439474476,5.0
2,4,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1573938898,4.0
3,5,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,830786155,4.0
4,7,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,835444730,4.0


In [7]:
dataFrame = dataFrame_without_ohe.copy()


In [8]:
# dataFrame = dataFrame.sample(n=250000, random_state=42)
df = dataFrame.copy()
print(df.shape)
df.head()

(25000095, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
0,1,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1147880044,5.0
1,3,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1439474476,5.0
2,4,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1573938898,4.0
3,5,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,830786155,4.0
4,7,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,835444730,4.0


## Ordenar el dataFrame por la columna 'timestamp' de mayor a menor, de la claifición mas reciente a la más antigua.


In [9]:
dataFrame_sorted_by_rating = dataFrame_without_ohe.copy()

# Ordenar el DataFrame por la columna 'timestamp' de mayor a menor
dataFrame_sorted_by_rating = dataFrame_sorted_by_rating.sort_values(by='timestamp', ascending=False)

## Se eliminan las filas que tengan NaN en las columnas **tag_by_user** o **tag_genome**

In [10]:
def contar_valores_nulos(df):
    # Cuenta los valores nulos en cada columna del DataFrame
    valores_nulos_por_columna = df.isnull().sum()
    # Cuenta los valores nulos en todo el DataFrame
    total_valores_nulos = df.isnull().sum().sum()
    # Imprime la cantidad de valores nulos por columna
    print("Valores nulos por columna:")
    print(valores_nulos_por_columna)
    # Imprime el total de valores nulos en el DataFrame
    print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

def eliminar_filas_nulas(df, columna):
    # Seleccionar las filas con valores nulos en la columna deseada
    filas_con_nulos = df[df[columna].isnull()]
    # Mostrar las filas con valores nulos
    # print("Filas con valores nulos en la columna", columna, ":")
    # print(filas_con_nulos)
    # Eliminar las filas con valores nulos en la columna deseada
    df = df.dropna(subset=[columna])
    return df

In [11]:
# contar_valores_nulos(df)
dataFrame_sorted_by_rating = eliminar_filas_nulas(dataFrame_sorted_by_rating, 'tag_genome')
# contar_valores_nulos(df)
dataFrame_sorted_by_rating = eliminar_filas_nulas(dataFrame_sorted_by_rating, 'tag_by_user')
# contar_valores_nulos(df)

In [12]:
print(dataFrame_sorted_by_rating.shape)
dataFrame_sorted_by_rating.head()

(197711, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
24669606,123527,201594,"country life, jump scares, farmhouse, horror, ...","chase, pg-13, original, fast paced, murder",Brightburn (2019),Drama|Horror|Sci-Fi|Thriller,1574316587,3.0
24390005,73725,190401,"Scotland, coverup, netflix, dark, Netflix orig...","original, mentor, catastrophe, silly fun, grea...",Calibre (2018),Thriller,1574300676,3.0
9039723,84470,171763,"musician, edgar wright, jamie foxx, blu-ray, c...","stylish, crime, heist, action, music",Baby Driver (2017),Action|Crime|Thriller,1574292524,3.0
22460648,61610,4036,"Udo Kier, vampire, 1920s, dracula, John Malkov...","vampire, immortality, weird, original, vampires",Shadow of the Vampire (2000),Drama|Horror,1574288547,4.0
23025020,34164,94070,"personal growth, characters, great old British...","indians, hotel, ensemble cast, feel-good, india","Best Exotic Marigold Hotel, The (2011)",Comedy|Drama,1574284807,3.5


## Se toma un subDataset del original

In [100]:
dataFrame = dataFrame_sorted_by_rating.copy()

n_samples = round(dataFrame.shape[0] * 0.05)
dataFrame = dataFrame.sample(n=n_samples, random_state=42)
df = dataFrame.copy()

In [101]:
# content_df = df[['title', 'genres', 'tag_by_user', 'tag_genome','rating']]
content_df = df[['movieId','title', 'genres', 'tag_by_user', 'tag_genome', 'timestamp','rating']]

In [102]:
print(content_df.shape)

(9886, 7)


## Funciones de limpieza de texto en Python

In [103]:
import re
import string

def remove_bars(text):
    """
    Elimina las barras verticales '|' del texto y las reemplaza por espacios.
    Parámetros:
    text (str): El texto de entrada que contiene barras verticales que se eliminarán.
    Retorna:
    str: El texto con las barras verticales eliminadas.
    """
    cleaned = re.sub(r'\|', ' ', text)
    return cleaned

def separate(text):
    """
    Divide el texto por comas, limpia cada parte eliminando el texto dentro de paréntesis, dígitos, espacios en blanco, puntuación, y lo convierte a minúsculas.
    Parámetros:
    text (str): El texto de entrada que contiene partes separadas por comas.
    Retorna:
    str: El texto limpiado y concatenado con cada parte separada por un espacio.
    """
    clean_text = []
    for t in text.split(','):
        cleaned = re.sub('\(.*\)', '', t) # Elimina el texto dentro de paréntesis
        cleaned = cleaned.translate(str.maketrans('','', string.digits))
        cleaned = cleaned.replace(' ', '')
        cleaned = cleaned.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text.append(cleaned)
    return ' '.join(clean_text)

def remove_punc(text):
    """
    Elimina la puntuación y los dígitos del texto y lo convierte a minúsculas.
    Parámetros:
    text (str): El texto de entrada que se limpiará.
    Retorna:
    str: El texto limpiado con la puntuación y los dígitos eliminados, convertido a minúsculas.
    """
    try:
        cleaned = text.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text = cleaned.translate(str.maketrans('','', string.digits))
    except Exception as e:
        print(f"ERROR -----------------------------------------> {e} AND {text}")
    return clean_text


In [104]:
# Ejemplo de uso: Pixar Animation Studios Warner Bros., Lancaster Gate
# text_with_bars = "oldie but goodie"
# text_with_bars = "Pixar Animation Studios"
text_with_bars = "Warner Bros., Lancaster Gate""""  """
cleaned_text = remove_punc(text_with_bars)
print(cleaned_text)

warner bros lancaster gate  


## Se eliminan signos de puntuacin de genres, tag_by_user, tag_genome y barras de genres

In [105]:
content_df_aux = content_df.copy()
content_df_aux['genres'] = (content_df_aux['genres'].apply(remove_bars)).apply(remove_punc)
content_df_aux['tag_by_user'] = content_df_aux['tag_by_user'].apply(remove_punc)
content_df_aux['tag_genome'] = content_df_aux['tag_genome'].apply(remove_punc)
# content_df_aux.set_index('title', inplace=True)
content_df_aux = content_df_aux.reset_index()

In [106]:
content_df = content_df_aux.copy()
content_df.head()

,index,movieId,title,genres,tag_by_user,tag_genome,timestamp,rating
0,8441598,99114,Django Unchained (2012),action drama western,th century friends bounty hunter love female c...,violence gratuitous violence imdb top violent...,1361057391,4.0
1,4892097,1732,"Big Lebowski, The (1998)",comedy crime,drugs best movie ever girls nudity full fronta...,quotable cult classic imdb top offbeat comedy...,1266336295,4.0
2,14023713,1947,West Side Story (1961),drama musical romance,movies every kid should seeentertainment week...,dance dancing music oscar best supporting acto...,1145634351,3.5
3,21893787,42011,Fun with Dick and Jane (2005),comedy crime,humiliation netflix riches to rags acidly funn...,comedy robbery funny remake silly fun,1138688114,2.0
4,5963555,5502,Signs (2002),horror scifi thriller,moral ambiguity what a twist dumb plot alien i...,alien aliens suspenseful suspense alien invasion,1282857065,3.0


## Se crea la bag of words con las columnas genres, tag_by_user y tag_genome

In [107]:
df_aux = content_df.copy()
df_aux = df_aux[['genres', 'tag_by_user', 'tag_genome']]
df_aux['bag_of_words'] = ''
df_aux.loc[:, 'bag_of_words'] = df_aux.loc[:, df_aux.columns[0:]].apply(lambda x: ' '.join(x), axis=1)
# content_df.set_index('original_title', inplace=True)
columnas_a_eliminar = ['tag_by_user', 'tag_genome']
# Elimina las columnas especificadas del DataFrame 'data'
content_df = content_df.drop(columnas_a_eliminar, axis=1)

content_df['genres'] = df_aux['bag_of_words']
content_df.rename(columns={'genres': 'bag_of_words'}, inplace=True)

content_df.head()

,index,movieId,title,bag_of_words,timestamp,rating
0,8441598,99114,Django Unchained (2012),action drama western th century friends bounty...,1361057391,4.0
1,4892097,1732,"Big Lebowski, The (1998)",comedy crime drugs best movie ever girls nudit...,1266336295,4.0
2,14023713,1947,West Side Story (1961),drama musical romance movies every kid should...,1145634351,3.5
3,21893787,42011,Fun with Dick and Jane (2005),comedy crime humiliation netflix riches to rag...,1138688114,2.0
4,5963555,5502,Signs (2002),horror scifi thriller moral ambiguity what a t...,1282857065,3.0


## Creando la Matriz TF-IDF Matrix

La matriz TF-IDF es una herramienta esencial en sistemas de recomendación basados en contenido, ya que ayuda a calcular la similitud entre elementos de contenido utilizando su contenido textual, lo que mejora la precisión de las recomendaciones al encontrar elementos que comparten características semánticas.

In [108]:
df = content_df.copy()

In [109]:
df["bag_of_words"].head()

0    action drama western th century friends bounty...
1    comedy crime drugs best movie ever girls nudit...
2    drama musical romance  movies every kid should...
3    comedy crime humiliation netflix riches to rag...
4    horror scifi thriller moral ambiguity what a t...
Name: bag_of_words, dtype: object

In [110]:
df["bag_of_words"].isnull().sum()

0

In [111]:
# Rellenemos el valor nulo de la variable cin bag_of_words con nada para evitar errores en los siguientes pasos
df['bag_of_words'] = df['bag_of_words'].fillna('')

In [112]:
df["bag_of_words"].isnull().sum()

0

In [113]:
# Convertir la colección de textos (en este caso, la bolsa de palabras)
# en una matriz TF-IDF.

# Vamos a eliminar las construcciones como a, an, the, and,
# pero que no tienen sentido para nosotros de nuestros DataFrames.
tfidf = TfidfVectorizer(stop_words="english")

In [114]:
# ajustar y transformar según el objeto tfidf
# Los de las filas son textos 'bolsa de palabras'. Los de las columnas son palabras únicas.

# Crear la matriz TF-IDF (Term Frequency-Inverse Document Frequency) a partir de la columna
# 'bag_of_words' del conjunto de datos. La matriz TF-IDF es una representación numérica
# de los documentos en función de la frecuencia de las palabras que contienen y de
# su importancia en el conjunto de documentos.
tfidf_matrix = tfidf.fit_transform(df['bag_of_words'])

In [115]:
# Cada fila representa un documento (en este caso, una película) y
# cada columna representa una palabra única en el conjunto de datos.

# Los valores en la matriz son las puntuaciones TF-IDF para cada palabra en cada documento.
tfidf_matrix.shape

(9886, 23602)

In [116]:
#Si queremos ver todas las palabras únicas de las columnas

# Filas: Cada fila de la matriz corresponde a una película en el conjunto de datos.
# Por ejemplo, si tienes 10,000 películas, habrá 10,000 filas en la matriz.

# Columnas: Cada columna representa una palabra única en el conjunto de datos.
# La cantidad de columnas es igual al número total de palabras únicas en todas
# las bolsas de palabras de películas.

# Valores: Los valores en la matriz son las puntuaciones TF-IDF para cada palabra en cada película.
# Estas puntuaciones miden la importancia relativa de una palabra en un documento en comparación
# con su frecuencia en el conjunto de documentos. Valores más altos indican que la palabra
# es más importante en el contexto de ese documento.

tfidf.get_feature_names_out()


array(['aa', 'aaliyah', 'aamir', ..., '独闯龙潭', '莫声版', '魔鬼司令'], dtype=object)

In [117]:
# tfidf scores
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Creación de la matriz de similitud del coseno

In [118]:
# calcula la similitud coseno entre todas las filas de la matriz TF-IDF.
# La similitud coseno mide el coseno del ángulo entre dos vectores y
# proporciona una medida de cuán similares son dos documentos.

# Calcula cos sim para todos los pares de documentos posibles uno por uno. En la matriz cosine_sim,
# cada película tiene similitudes entre sí

if IN_COLAB and GPU:
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)

    import cupy as cp
    # Convertir la matriz TF-IDF a cupy array
    tfidf_matrix_cupy = cp.array(tfidf_matrix.toarray())
    # Calcular la similitud del coseno utilizando cupy.matmul
    cosine_sim_cupy = cp.matmul(tfidf_matrix_cupy, tfidf_matrix_cupy.T)
    cosine_sim = cosine_sim_cupy
else:
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

### OTRA FORMA DE HACER LA SMILITUD DEL COSENO

Calcula el producto punto (producto interno) entre pares de vectores o matrices de vectores.
En el contexto de matrices, como la matriz TF-IDF, linear_kernel calcula el producto punto entre cada par de filas (documentos) en la matriz.

In [119]:
# from sklearn.metrics.pairwise import linear_kernel
# linear_similarity_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)
# linear_similarity_matrix.shape


In [120]:
# cosine_sim[i, j] representa la similitud coseno entre la película con el índice i y
# la película con el índice j en el conjunto de datos.

cosine_sim.shape

(9886, 9886)

In [121]:
cosine_sim

array([[1.        , 0.06449652, 0.17080229, ..., 0.0412217 , 0.0295986 ,
        0.01326478],
       [0.06449652, 1.        , 0.0232609 , ..., 0.01973442, 0.02416497,
        0.01272072],
       [0.17080229, 0.0232609 , 1.        , ..., 0.03474215, 0.08005978,
        0.01685395],
       ...,
       [0.0412217 , 0.01973442, 0.03474215, ..., 1.        , 0.06499389,
        0.01238806],
       [0.0295986 , 0.02416497, 0.08005978, ..., 0.06499389, 1.        ,
        0.04909974],
       [0.01326478, 0.01272072, 0.01685395, ..., 0.01238806, 0.04909974,
        1.        ]])

In [122]:
cosine_sim[1].shape

(9886,)

# Recomendaciones basada en la similitud del coseno

In [123]:
# Vamos a crear una serie pd de índices y nombres de películas
# se crea una Serie de Pandas llamada indices donde los índices son
# los títulos de las películas y los valores son los índices originales del DataFrame df.
indices = pd.Series(df.index, index=df['title'])

In [124]:
indices.head()

title
Django Unchained (2012)          0
Big Lebowski, The (1998)         1
West Side Story (1961)           2
Fun with Dick and Jane (2005)    3
Signs (2002)                     4
dtype: int64

In [125]:
indices.shape

(9886,)

In [126]:
# contemos la información del índice de las películas y simplifiquemos las películas más repetitivas a las más recientes
indices.index.value_counts().head()

Star Wars: Episode IV - A New Hope (1977)    78
Pulp Fiction (1994)                          44
Silence of the Lambs, The (1991)             36
Interstellar (2014)                          36
Forrest Gump (1994)                          34
Name: title, dtype: int64

### Se eliminan las películas duplicadas del índice

In [127]:
# Se eliminan las películas duplicadas del índice, conservando solo la última aparición de cada película.
# Esto se hace para evitar ambigüedades y asegurar que cada película tenga un índice único.
indices = indices[~indices.index.duplicated(keep='last')]

In [128]:
indices.index.value_counts().head()

Creature from the Black Lagoon, The (1954)      1
Anniversary Party, The (2001)                   1
Lilies of the Field (1963)                      1
Men in Black III (M.III.B.) (M.I.B.³) (2012)    1
Paddington (2014)                               1
Name: title, dtype: int64

In [129]:
indices.shape

(4056,)

In [130]:
indices

title
Creature from the Black Lagoon, The (1954)       6
Longest Day, The (1962)                          7
Philosophers, The (After The Dark) (2013)       10
Murder in the First (1995)                      14
Snow White and the Huntsman (2012)              16
                                              ... 
Carrie (2013)                                 9881
Social Network, The (2010)                    9882
Emma (1996)                                   9883
Prime (2005)                                  9884
Conspiracy Theory (1997)                      9885
Length: 4056, dtype: int64

### Pelicula a recomendar

In [131]:
movie_title = "Wolf of Wall Street, The (2013)"

In [132]:
# Asigno el índice de la película "Wolf of Wall Street, The (2013)" a la variable
movie_index = indices[movie_title]
movie_index

8536

### Para ver en qué se parece la película anterior a las demás películas

In [133]:
cosine_sim[movie_index]

array([0.11629347, 0.07459687, 0.10180035, ..., 0.02985216, 0.03255098,
       0.01564922])

In [134]:
cosine_sim[movie_index].shape

(9886,)

# Función de recomendación basada en contenido, dada una pelicula.

In [135]:
def content_based_recommender(title, cosine_sim, dataframe, num_recomendations=10):
    """
    Genera recomendaciones basadas en contenido para una película específica.
    Parameters:
    title (str): El título de la película para la cual se generarán las recomendaciones.
    cosine_sim (array-like): Matriz de similitud coseno entre películas.
    dataframe (DataFrame): DataFrame que contiene la información de las películas.
    num_recomendations (int): El número de recomendaciones a generar (por defecto es 10).
    Returns:
    Series: Una Serie de Pandas que contiene los títulos de las películas recomendadas.
    """
    # Crear índices basados en los títulos de las películas
    indices = pd.Series(dataframe.index, index=dataframe['title'])
    indices = indices[~indices.index.duplicated(keep='last')]  # Eliminar índices duplicados

    # Capturar el índice de la película seleccionada
    movie_index = indices[title]

    # Calcular los puntajes de similitud basados en el título
    if IN_COLAB and GPU:
        similarity_scores = pd.DataFrame(cosine_sim[movie_index].get(), columns=["score"])
    else:
        similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])

    # Ordenar los índices de las películas por puntaje de similitud
    movie_indices_sort = similarity_scores.sort_values("score", ascending=False).index

    # Eliminar títulos duplicados y excluir la película seleccionada
    duplicated_titles = dataframe.loc[movie_indices_sort, 'title'].duplicated(keep='first')
    indices_sin_duplicados = movie_indices_sort[~duplicated_titles]
    filas_sin_duplicados_y_exclusion = dataframe.loc[indices_sin_duplicados]
    filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != title]

    # Devolver los títulos de las películas recomendadas
    return filas_sin_duplicados_y_exclusion['title'].head(num_recomendations)


In [136]:
content_based_recommender("Wolf of Wall Street, The (2013)", cosine_sim, df)

884                          Too Big to Fail (2011)
6454                             Wall Street (1987)
3792                             Margin Call (2011)
3768                          Big Short, The (2015)
88                               Boiler Room (2000)
3055                              Inside Job (2010)
5353    Enron: The Smartest Guys in the Room (2005)
2389         Wall Street: Money Never Sleeps (2010)
2267                                 Traffic (2000)
3473                              Goodfellas (1990)
Name: title, dtype: object

In [137]:
content_based_recommender("Matrix, The (1999)", cosine_sim, df)

6037                  Matrix Reloaded, The (2003)
5710               Matrix Revolutions, The (2003)
2190    Star Wars: Episode IV - A New Hope (1977)
3074            Terminator 2: Judgment Day (1991)
7051                       Terminator, The (1984)
2086                                  Tron (1982)
6863                         Jurassic Park (1993)
3267                                Avatar (2009)
4330                           Cloud Atlas (2012)
9466                             Inception (2010)
Name: title, dtype: object

In [138]:
content_based_recommender("Star Wars: Episode IV - A New Hope (1977)", cosine_sim, df)

8645    Star Wars: Episode VI - Return of the Jedi (1983)
5336    Star Wars: Episode V - The Empire Strikes Back...
3640                                        Aliens (1986)
1505                       Approaching the Unknown (2016)
768                          2001: A Space Odyssey (1968)
7651                                Wing Commander (1999)
8401                                      Serenity (2005)
9668                                         Alien (1979)
3507     Star Wars: Episode I - The Phantom Menace (1999)
2272                       Solo: A Star Wars Story (2018)
Name: title, dtype: object

In [139]:
content_based_recommender('Pulp Fiction (1994)', cosine_sim, df)

2981       Reservoir Dogs (1992)
9237         Jackie Brown (1997)
6696    Kill Bill: Vol. 2 (2004)
432     Kill Bill: Vol. 1 (2003)
9561             Sin City (2005)
3473           Goodfellas (1990)
5398       Godfather, The (1972)
3327         True Romance (1993)
155      Django Unchained (2012)
4784         Forrest Gump (1994)
Name: title, dtype: object

In [140]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def generate_user_recommendations(user_id, user_ratings, movies_features, cosine_sim_matrix, num_recommendations=10):
    """
    Genera recomendaciones de películas para un usuario basado en sus valoraciones positivas.

    Parameters:
    user_id (int): ID del usuario para el cual generar recomendaciones.
    user_ratings (DataFrame): DataFrame que contiene las valoraciones de los usuarios.
    movies_features (DataFrame): DataFrame con las características de las películas.
    cosine_sim_matrix (array-like): Matriz de similitud coseno entre películas.
    num_recommendations (int): Número de recomendaciones a generar.

    Returns:
    list: Lista de títulos de películas recomendadas.
    """
    # Filtrar películas que el usuario ha valorado positivamente
    liked_movies = user_ratings[(user_ratings['userId'] == user_id) & (user_ratings['rating'] > 3.5)]

    # Si el usuario no ha valorado suficientes películas positivamente, considera todas sus valoraciones
    if len(liked_movies) < 3:
        liked_movies = user_ratings[user_ratings['userId'] == user_id]

    # Crear el perfil del usuario basado en las características de las películas que le gustan
    user_profile = np.mean(movies_features.loc[liked_movies['movieId']], axis=0)

    # Calcular la similitud del perfil del usuario con todas las películas
    similarity_scores = cosine_similarity([user_profile], movies_features)[0]

    # Obtener los índices de las películas con las puntuaciones más altas
    recommended_movie_indices = similarity_scores.argsort()[-num_recommendations:][::-1]

    # Filtrar las películas que el usuario ya ha visto
    already_watched = liked_movies['movieId'].tolist()
    recommendations = [idx for idx in recommended_movie_indices if idx not in already_watched][:num_recommendations]

    # Devolver los títulos de las películas recomendadas
    recommended_titles = movies_features.iloc[recommendations]['title'].tolist()

    return recommended_titles


In [145]:
# Seleccionar un userId aleatorio de todos los userId únicos en el DataFrame
user_id_random = np.random.choice(dataFrame['userId'].unique())

# Mostrar las filas del DataFrame correspondientes a ese userId aleatorio
user_data = dataFrame[dataFrame['userId'] == user_id_random]


In [142]:
user_rating = user_data['rating']

In [146]:
user_data

,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
22758691,160540,6944,"fool, overly protective father, family gatheri...","comedy, remake, wedding, girlie movie, family",Father of the Bride (1991),Comedy,1145171359,2.5
15479829,160540,71579,"rebellion, dumb blonde, bad message, school, P...","coming of age, london, adolescence, school, 1960s","Education, An (2009)",Drama|Romance,1280587597,4.0
11740468,160540,1258,"insane, jack nicholson, dvd, bartender, overra...","masterpiece, insanity, imdb top 250, creepy, a...","Shining, The (1980)",Horror,884283537,3.0
24023232,160540,7020,"liz should see, blind man, Jocelyn Moorhouse, ...","character study, obsession, relationships, lon...",Proof (1991),Comedy|Drama|Romance,1124606487,4.5
13602465,160540,1057,"christmas eve, restaurant, tourist, Paris, sen...","original, new york, music, musical, good music",Everyone Says I Love You (1996),Comedy|Musical|Romance,1372793579,3.5
22566211,160540,7450,"romance, predictable, Pierce Brosnan, fire bet...","romantic comedy, good romantic comedies, girli...",Laws of Attraction (2004),Comedy|Romance,1106889351,1.5
15231150,160540,6218,"immigrants, girls, love, thought-provoking, sp...","culture clash, soccer, girlie movie, feel-good...",Bend It Like Beckham (2002),Comedy|Drama|Romance,1073142831,3.5
18796182,160540,2541,"Soul searching, KAF, Nudity (Topless - Brief),...","teens, teen movie, sexy, high school, teen",Cruel Intentions (1999),Drama,921602438,3.0
12982176,160540,1962,"slow plot, chauffeur, 1950s, car accident, sta...","oscar, segregation, friendship, unlikely frien...",Driving Miss Daisy (1989),Drama,1088084044,3.0
16601967,160540,8623,"strong performances, Fred Schepisi, related:Cy...","comedy, romantic comedy, sweet, based on a pla...",Roxanne (1987),Comedy|Romance,1088788684,4.5


In [147]:
# Ejemplo de uso
user_id = user_id_random  # ID del usuario para el cual generar recomendaciones
user_ratings = user_data  # Tu DataFrame de valoraciones de usuarios
movies_features = df  # Tu DataFrame de características de películas
cosine_sim_matrix = cosine_sim  # Tu matriz de similitud coseno entre películas

recommendations = generate_user_recommendations(user_id, user_ratings, movies_features, cosine_sim_matrix)
print(recommendations)

KeyError: '[71579, 138036, 155874, 55444, 80126, 68358, 27790, 55851] not in index'

# Función de recomendación basada en contenido, dada una pelicula.

In [ ]:
# Para un usario se puede medir, pasandole el usario, y de las peliculas que ha visto ir sacando la recomendación por pelicula e ir evaluando el modelo porpelicula.

In [ ]:
def content_based_recommender_for_user(user_id, cosine_sim, completeDataframe, df_bag_of_words, num_recommendations=10):
    """
    Genera recomendaciones basadas en contenido para un usuario específico.

    Parameters:
    user_id (int): El ID del usuario para el cual se generarán las recomendaciones.
    cosine_sim (array-like): Matriz de similitud coseno entre películas.
    dataframe (DataFrame): DataFrame que contiene la información de las películas.
    num_recomendations (int): El número de recomendaciones a generar (por defecto es 10).

    Returns:
    Series: Una Serie de Pandas que contiene los títulos de las películas recomendadas.
    """
    # Obtener las películas que el usuario ha visto o calificado previamente
    peliculas_vistas_usario = completeDataframe[completeDataframe['userId'] == user_id]
    peliculas_vistas_usario.dropna(subset=['title'])
    titulo_peliculas_vistas = peliculas_vistas_usario['title']
    print(f'\n Peliculas vistas por el usuario {user_id} --> \n{titulo_peliculas_vistas}\n')

    for titulo_pelicula in titulo_peliculas_vistas:
        print(f"\n Peliculas recomendadas según: {titulo_pelicula}: \n")
        peliculas_recomendadas = content_based_recommender(titulo_pelicula, cosine_sim, df_bag_of_words, num_recommendations)
        print(f"{peliculas_recomendadas}: \n\n")
        # En este punto hay que evaluar las recomendaciones (filas_sin_duplicados_y_exclusion) con las peliculas vistas por el usario (titulo_peliculas_vistas)

In [ ]:
# Recomendar películas para el usuario con ID 123
num_recommendations = 10
dfCopy= dataFrame.copy()
recommended_movies = content_based_recommender_for_user(dfCopy['userId'].sample().iloc[0], cosine_sim, dataFrame,df, num_recommendations)
print(recommended_movies)

#### Creación de una columna nueva con los valores de similitud de una pelicula con el resto de peliculas.

In [ ]:
# # Veamos los Smilarity Scores que expresan las similitudes entre la película "Wolf of Wall Street, The (2013)" y otras películas
# if IN_COLAB:
#     similarity_scores = pd.DataFrame(cosine_sim[movie_index].get(), columns=["score"])
# else:
#     similarity_scores = pd.DataFrame(cosine_sim[movie_index],
#                                     columns=["score"])

In [ ]:
# # Las similitudes entre la película "Wolf of Wall Street, The (2013)" y el resto de películas
# similarity_scores.head()

In [ ]:
# similarity_scores.shape

###  Ordena los índices de las películas en función de sus puntajes de similitud, de mayor a menor.

In [ ]:
# movie_indices_sort = similarity_scores.sort_values("score", ascending=False).index

In [ ]:
# import pandas as pd

# # Supongamos que tu DataFrame se llama df y la Serie de índices es index_series
# # index_series = Int64Index([9859, 3228, 1656, 8951, 6811, 998, 5030, 6972, 8591, 4820, ...])

# # Paso 1
# duplicated_titles = df.loc[movie_indices_sort, 'title'].duplicated(keep='first')

# # Paso 2
# indices_sin_duplicados = movie_indices_sort[~duplicated_titles]

# # Paso 3
# filas_sin_duplicados = df.loc[indices_sin_duplicados]

# # Ahora filas_sin_duplicados contiene las filas correspondientes a los índices
# # en index_series, eliminando las filas con títulos duplicados, manteniendo la primera ocurrencia.

# # Supongamos que el string que deseas excluir es 'The Promise'
# string_a_excluir = movie_title

# # Filtrar las filas que no contienen el string a excluir en la columna 'title'
# filas_sin_duplicados_y_exclusion = df.loc[indices_sin_duplicados]
# filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != string_a_excluir]

In [ ]:
# # Supongamos que el string que deseas excluir es 'The Promise'
# string_a_excluir = 'Wolf of Wall Street, The (2013)'

# # Filtrar las filas que no contienen el string a excluir en la columna 'title'
# filas_sin_duplicados_y_exclusion = df.loc[indices_sin_duplicados]
# filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != string_a_excluir]


In [ ]:
# filas_sin_duplicados_y_exclusion.head()

In [ ]:
# Vamos a listar las puntuaciones de similitud de la película 'Sherlock Holmes' en orden descendente. Empieza por 1 porque es la primera película.
# movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index

In [ ]:
# Go to the indexes we selected in our first data set
# df['title'].iloc[movie_indices]

In [ ]:
# def calculate_cosine_sim(dataframe):
#     tfidf = TfidfVectorizer(stop_words='english')
#     dataframe['bag_of_words'] = dataframe['bag_of_words'].fillna('')
#     tfidf_matrix = tfidf.fit_transform(dataframe['bag_of_words'])
#     cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
#     return cosine_sim

In [ ]:
# cosine_sim = calculate_cosine_sim(df)
# content_based_recommender('Pulp Fiction (1994)', cosine_sim, df)